In [1]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [9]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.compat.v1.Session(graph= tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

Using TensorFlow backend.


In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [15]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [24]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.Session object at 0x000001FA6354C630>>
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\tensorflow_core\python\client\session.py", line 761, in __del__
    if self._session is not None:
AttributeError: 'Session' object has no attribute '_session'


In [25]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [26]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 64s 9ms/step - loss: 1.2773 - accuracy: 0.4693 - val_loss: 1.0427 - val_accuracy: 0.5741
Epoch 2/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.9190 - accuracy: 0.6035 - val_loss: 0.9349 - val_accuracy: 0.5677
Epoch 3/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.7878 - accuracy: 0.6390 - val_loss: 0.7974 - val_accuracy: 0.6098
Epoch 4/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.7117 - accuracy: 0.6517 - val_loss: 0.7804 - val_accuracy: 0.6132
Epoch 5/30
7352/7352 [==============================] - 65s 9ms/step - loss: 0.6653 - accuracy: 0.6598 - val_loss: 0.7542 - val_accuracy: 0.6230
Epoch 6/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.6326 - accuracy: 0.6632 - val_loss: 0.7230 - val_accuracy: 0.6183
Epoch 7/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.596

In [27]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        10        0                   0   
SITTING                  3      378       110        0                   0   
STANDING                 0       69       462        1                   0   
WALKING                  0        0         2      456                  25   
WALKING_DOWNSTAIRS       0        0         0        0                 416   
WALKING_UPSTAIRS         0        0         2        1                  19   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            17  
SITTING                            0  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                 4  
WALKING_UPSTAIRS                 449  


In [28]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [29]:
score

[0.4293035353590176, 0.9063454270362854]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

# ASSIGNMENT

In [43]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 63s 9ms/step - loss: 1.3283 - accuracy: 0.4455 - val_loss: 1.2288 - val_accuracy: 0.4455
Epoch 2/30
7352/7352 [==============================] - 63s 9ms/step - loss: 1.1613 - accuracy: 0.4951 - val_loss: 1.0449 - val_accuracy: 0.5718
Epoch 3/30
7352/7352 [==============================] - 63s 9ms/step - loss: 1.0198 - accuracy: 0.5180 - val_loss: 1.1316 - val_accuracy: 0.5765
Epoch 4/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.8286 - accuracy: 0.6193 - val_loss: 0.7495 - val_accuracy: 0.6278
Epoch 5/30
7352/7352 [==============================] - 63s 9ms/step - loss: 0.7025 - accuracy: 0.6598 - val_loss: 0.7118 - val_accuracy: 0.6322
Epoch 6/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.6752 - accuracy: 0.6609 - val_loss: 0.7598 - val_accuracy: 0.6328
Epoch 7/30
7352/7352 [==============================] - 64s 9ms/step - loss: 0.640

In [46]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      379        88        0                   0   
STANDING                 0       83       445        2                   0   
WALKING                  0        0         0      454                  37   
WALKING_DOWNSTAIRS       0        0         0        2                 409   
WALKING_UPSTAIRS         0        1         1       31                  25   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                           24  
STANDING                           2  
WALKING                            5  
WALKING_DOWNSTAIRS                 9  
WALKING_UPSTAIRS                 413  


In [47]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [48]:
score

[0.3594353889522391, 0.894808292388916]

In [68]:
len(X_train[1])

128

In [84]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.75))
model.add(LSTM(32,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.75))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_48 (LSTM)               (None, 128, 32)           5376      
_________________________________________________________________
dropout_32 (Dropout)         (None, 128, 32)           0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_33 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 6)                 198       
Total params: 13,894
Trainable params: 13,894
Non-trainable params: 0
_________________________________________________________________


In [85]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [86]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='accuracy',patience=2)
callbacks = [early_stop]

In [87]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 126s 17ms/step - loss: 1.4905 - accuracy: 0.3984 - val_loss: 1.3451 - val_accuracy: 0.3482
Epoch 2/30
7352/7352 [==============================] - 126s 17ms/step - loss: 1.1664 - accuracy: 0.4830 - val_loss: 0.9621 - val_accuracy: 0.5755
Epoch 3/30
7352/7352 [==============================] - 126s 17ms/step - loss: 0.9819 - accuracy: 0.5486 - val_loss: 0.8352 - val_accuracy: 0.6899
Epoch 4/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.8936 - accuracy: 0.5711 - val_loss: 0.7979 - val_accuracy: 0.6447
Epoch 5/30
7352/7352 [==============================] - 126s 17ms/step - loss: 0.9400 - accuracy: 0.5458 - val_loss: 0.8455 - val_accuracy: 0.5463
Epoch 6/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.8755 - accuracy: 0.5720 - val_loss: 0.7854 - val_accuracy: 0.5840
Epoch 7/30
7352/7352 [==============================] - 126s 17ms/step

In [91]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(20,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(20,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 128, 20)           2400      
_________________________________________________________________
dropout_36 (Dropout)         (None, 128, 20)           0         
_________________________________________________________________
lstm_53 (LSTM)               (None, 20)                3280      
_________________________________________________________________
dropout_37 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 126       
Total params: 5,806
Trainable params: 5,806
Non-trainable params: 0
_________________________________________________________________


In [92]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [93]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='accuracy',patience=2)
callbacks = [early_stop]

In [94]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 122s 17ms/step - loss: 1.2968 - accuracy: 0.5180 - val_loss: 0.9934 - val_accuracy: 0.6295
Epoch 2/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.9169 - accuracy: 0.6438 - val_loss: 0.7864 - val_accuracy: 0.7282
Epoch 3/30
7352/7352 [==============================] - 121s 16ms/step - loss: 0.7902 - accuracy: 0.7095 - val_loss: 0.6773 - val_accuracy: 0.7357
Epoch 4/30
7352/7352 [==============================] - 121s 16ms/step - loss: 0.7136 - accuracy: 0.7330 - val_loss: 0.6054 - val_accuracy: 0.7849
Epoch 5/30
7352/7352 [==============================] - 123s 17ms/step - loss: 0.6720 - accuracy: 0.7591 - val_loss: 0.6572 - val_accuracy: 0.7645
Epoch 6/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.5740 - accuracy: 0.8332 - val_loss: 0.4878 - val_accuracy: 0.8449
Epoch 7/30
7352/7352 [==============================] - 122s 17ms/step

In [95]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      407        62       22                   0   
STANDING                 1       96       432        3                   0   
WALKING                  0        0         1      453                  37   
WALKING_DOWNSTAIRS       0        0         0       11                 408   
WALKING_UPSTAIRS         0        0         0       16                  28   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            5  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 427  


In [96]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 11s 4ms/step


[0.4682972089535494, 0.9039701223373413]

In [97]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(24,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(24,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_54 (LSTM)               (None, 128, 24)           3264      
_________________________________________________________________
dropout_38 (Dropout)         (None, 128, 24)           0         
_________________________________________________________________
lstm_55 (LSTM)               (None, 24)                4704      
_________________________________________________________________
dropout_39 (Dropout)         (None, 24)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 150       
Total params: 8,118
Trainable params: 8,118
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [99]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [early_stop]

In [100]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 126s 17ms/step - loss: 1.2037 - accuracy: 0.5173 - val_loss: 0.9069 - val_accuracy: 0.5718
Epoch 2/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.8513 - accuracy: 0.6268 - val_loss: 0.7921 - val_accuracy: 0.6003
Epoch 3/30
7352/7352 [==============================] - 126s 17ms/step - loss: 0.7388 - accuracy: 0.6911 - val_loss: 0.7528 - val_accuracy: 0.6851
Epoch 4/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.6451 - accuracy: 0.7440 - val_loss: 0.6266 - val_accuracy: 0.7333
Epoch 5/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.5591 - accuracy: 0.7775 - val_loss: 0.6781 - val_accuracy: 0.7214
Epoch 6/30
7352/7352 [==============================] - 126s 17ms/step - loss: 0.4845 - accuracy: 0.8033 - val_loss: 0.5954 - val_accuracy: 0.7631
Epoch 7/30
7352/7352 [==============================] - 125s 17ms/step

In [101]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 536        0         0        0                   0   
SITTING                  6      353       116        2                   0   
STANDING                 0       74       451        4                   0   
WALKING                  0        0         0      452                  32   
WALKING_DOWNSTAIRS       0        0         0        2                 418   
WALKING_UPSTAIRS         0        0         0       15                   9   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             1  
SITTING                           14  
STANDING                           3  
WALKING                           12  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 447  


In [102]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 19s 7ms/step


[0.38118069241371727, 0.9015948176383972]

In [103]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(28,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(28,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_56 (LSTM)               (None, 128, 28)           4256      
_________________________________________________________________
dropout_40 (Dropout)         (None, 128, 28)           0         
_________________________________________________________________
lstm_57 (LSTM)               (None, 28)                6384      
_________________________________________________________________
dropout_41 (Dropout)         (None, 28)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 174       
Total params: 10,814
Trainable params: 10,814
Non-trainable params: 0
_________________________________________________________________


In [104]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [105]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [early_stop]

In [106]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 214s 29ms/step - loss: 1.2012 - accuracy: 0.5029 - val_loss: 1.0125 - val_accuracy: 0.4425
Epoch 2/30
7352/7352 [==============================] - 212s 29ms/step - loss: 0.8261 - accuracy: 0.6285 - val_loss: 0.7214 - val_accuracy: 0.6216
Epoch 3/30
7352/7352 [==============================] - 209s 28ms/step - loss: 0.6916 - accuracy: 0.6912 - val_loss: 0.9501 - val_accuracy: 0.6250
Epoch 4/30
7352/7352 [==============================] - 214s 29ms/step - loss: 0.6101 - accuracy: 0.7364 - val_loss: 0.5421 - val_accuracy: 0.7655
Epoch 5/30
7352/7352 [==============================] - 229s 31ms/step - loss: 0.5116 - accuracy: 0.7809 - val_loss: 0.5140 - val_accuracy: 0.7635
Epoch 6/30
7352/7352 [==============================] - 258s 35ms/step - loss: 0.4753 - accuracy: 0.7968 - val_loss: 0.7294 - val_accuracy: 0.7234


In [107]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(28,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.75))
model.add(LSTM(28,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.75))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_58 (LSTM)               (None, 128, 28)           4256      
_________________________________________________________________
dropout_42 (Dropout)         (None, 128, 28)           0         
_________________________________________________________________
lstm_59 (LSTM)               (None, 28)                6384      
_________________________________________________________________
dropout_43 (Dropout)         (None, 28)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 174       
Total params: 10,814
Trainable params: 10,814
Non-trainable params: 0
_________________________________________________________________


In [108]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [109]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [early_stop]

In [110]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 122s 17ms/step - loss: 1.3003 - accuracy: 0.4597 - val_loss: 0.9513 - val_accuracy: 0.5375
Epoch 2/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.9403 - accuracy: 0.5325 - val_loss: 0.8734 - val_accuracy: 0.5182
Epoch 3/30
7352/7352 [==============================] - 121s 17ms/step - loss: 0.8687 - accuracy: 0.5355 - val_loss: 0.8599 - val_accuracy: 0.5205


In [111]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(28,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.25))
model.add(LSTM(28,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_60 (LSTM)               (None, 128, 28)           4256      
_________________________________________________________________
dropout_44 (Dropout)         (None, 128, 28)           0         
_________________________________________________________________
lstm_61 (LSTM)               (None, 28)                6384      
_________________________________________________________________
dropout_45 (Dropout)         (None, 28)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 6)                 174       
Total params: 10,814
Trainable params: 10,814
Non-trainable params: 0
_________________________________________________________________


In [112]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [113]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [early_stop]

In [114]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 122s 17ms/step - loss: 1.1266 - accuracy: 0.5020 - val_loss: 0.9131 - val_accuracy: 0.5460
Epoch 2/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.7529 - accuracy: 0.6326 - val_loss: 0.7405 - val_accuracy: 0.6342
Epoch 3/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.6505 - accuracy: 0.6881 - val_loss: 0.7630 - val_accuracy: 0.6658
Epoch 4/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.5036 - accuracy: 0.7799 - val_loss: 0.5538 - val_accuracy: 0.7686
Epoch 5/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.4192 - accuracy: 0.8275 - val_loss: 0.4469 - val_accuracy: 0.8398
Epoch 6/30
7352/7352 [==============================] - 122s 17ms/step - loss: 0.3390 - accuracy: 0.8791 - val_loss: 0.3812 - val_accuracy: 0.8633
Epoch 7/30
7352/7352 [==============================] - 123s 17ms/step

In [115]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      370        96       17                   2   
STANDING                 0       61       468        3                   0   
WALKING                  0        0         0      468                  26   
WALKING_DOWNSTAIRS       0        0         0        3                 416   
WALKING_UPSTAIRS         0        1         1        8                  46   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            4  
STANDING                           0  
WALKING                            2  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 415  


In [116]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 12s 4ms/step


[0.4081978703499752, 0.9073634147644043]

In [16]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.25))
model.add(LSTM(32,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 31,558
Trainable params: 31,558
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [18]:
from keras.callbacks.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [early_stop]

In [19]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 144s 20ms/step - loss: 1.1056 - accuracy: 0.5331 - val_loss: 0.8742 - val_accuracy: 0.6356
Epoch 2/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.6890 - accuracy: 0.7325 - val_loss: 0.6520 - val_accuracy: 0.7615
Epoch 3/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.4847 - accuracy: 0.8312 - val_loss: 0.4826 - val_accuracy: 0.8500
Epoch 4/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.2758 - accuracy: 0.9173 - val_loss: 0.4425 - val_accuracy: 0.8643
Epoch 5/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.2132 - accuracy: 0.9347 - val_loss: 0.3679 - val_accuracy: 0.8904
Epoch 6/30
7352/7352 [==============================] - 141s 19ms/step - loss: 0.1813 - accuracy: 0.9436 - val_loss: 0.4470 - val_accuracy: 0.8860
Epoch 7/30
7352/7352 [==============================] - 140s 19ms/step

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      375       112        0                   0   
STANDING                 0       55       477        0                   0   
WALKING                  0        0        15      424                  46   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         3       13                  23   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                           11  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 432  


In [21]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 14s 5ms/step


[0.41741838527891234, 0.9043094515800476]

In [22]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(32,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 31,558
Trainable params: 31,558
Non-trainable params: 0
_________________________________________________________________


In [25]:
from keras.callbacks.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint_1 = ModelCheckpoint("model_1.1", monitor="val_accuracy",mode="max",save_best_only = True, verbose=1) 


early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [checkpoint_1,early_stop]

In [26]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [27]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 141s 19ms/step - loss: 1.1688 - accuracy: 0.5530 - val_loss: 0.8774 - val_accuracy: 0.6518

Epoch 00001: val_accuracy improved from -inf to 0.65185, saving model to model_1.1
Epoch 2/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.7562 - accuracy: 0.6999 - val_loss: 0.6272 - val_accuracy: 0.7289

Epoch 00002: val_accuracy improved from 0.65185 to 0.72888, saving model to model_1.1
Epoch 3/30
7352/7352 [==============================] - 141s 19ms/step - loss: 0.6180 - accuracy: 0.7519 - val_loss: 0.6287 - val_accuracy: 0.7387

Epoch 00003: val_accuracy improved from 0.72888 to 0.73872, saving model to model_1.1
Epoch 4/30
7352/7352 [==============================] - 141s 19ms/step - loss: 0.5084 - accuracy: 0.8173 - val_loss: 0.4337 - val_accuracy: 0.8324

Epoch 00004: val_accuracy improved from 0.73872 to 0.83237, saving model to model_1.1
Epoch 5/30
7352/735

In [31]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         3        0                   0   
SITTING                  0      375       112        0                   0   
STANDING                 0       74       456        2                   0   
WALKING                  0        0         0      463                   0   
WALKING_DOWNSTAIRS       0        0         0       19                 350   
WALKING_UPSTAIRS         0        0         0       20                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            24  
SITTING                            4  
STANDING                           0  
WALKING                           33  
WALKING_DOWNSTAIRS                51  
WALKING_UPSTAIRS                 451  


In [34]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 15s 5ms/step


[0.5100547778261941, 0.8839497566223145]

In [35]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.75))
model.add(LSTM(32,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.75))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 31,558
Trainable params: 31,558
Non-trainable params: 0
_________________________________________________________________


In [36]:
from keras.callbacks.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint_1 = ModelCheckpoint("model_1.1", monitor="val_accuracy",mode="max",save_best_only = True, verbose=1) 


early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [checkpoint_1,early_stop]

In [37]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [38]:
#Training the model
LSTM=model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 141s 19ms/step - loss: 1.3000 - accuracy: 0.4868 - val_loss: 0.9620 - val_accuracy: 0.5911

Epoch 00001: val_accuracy improved from -inf to 0.59111, saving model to model_1.1
Epoch 2/30
7352/7352 [==============================] - 141s 19ms/step - loss: 0.9806 - accuracy: 0.5770 - val_loss: 0.8821 - val_accuracy: 0.5857

Epoch 00002: val_accuracy did not improve from 0.59111
Epoch 3/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.8653 - accuracy: 0.6137 - val_loss: 0.8124 - val_accuracy: 0.6077

Epoch 00003: val_accuracy improved from 0.59111 to 0.60774, saving model to model_1.1
Epoch 4/30
7352/7352 [==============================] - 140s 19ms/step - loss: 0.8039 - accuracy: 0.6249 - val_loss: 0.7777 - val_accuracy: 0.6196

Epoch 00004: val_accuracy improved from 0.60774 to 0.61961, saving model to model_1.1
Epoch 5/30
7352/7352 [============================

In [40]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      418        69        1                   0   
STANDING                 0      114       406        1                   0   
WALKING                  0        0         0      449                   2   
WALKING_DOWNSTAIRS       0        0         0       25                 346   
WALKING_UPSTAIRS         0        0         0       18                   3   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            3  
STANDING                          11  
WALKING                           45  
WALKING_DOWNSTAIRS                49  
WALKING_UPSTAIRS                 450  


In [41]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 16s 5ms/step


[0.622152369769943, 0.8751272559165955]

In [20]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64,return_sequences = True,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.6))
model.add(LSTM(64,return_sequences = False,input_shape=(128,9)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________


In [21]:
from keras.callbacks.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint_1 = ModelCheckpoint("model_1.1", monitor="val_accuracy",mode="max",save_best_only = True, verbose=1) 


early_stop=EarlyStopping(monitor='val_accuracy',patience=2)
callbacks = [checkpoint_1,early_stop]

In [22]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [23]:
#Training the model
lstm=model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 228s 31ms/step - loss: 1.0310 - accuracy: 0.5638 - val_loss: 0.7878 - val_accuracy: 0.6614

Epoch 00001: val_accuracy improved from -inf to 0.66135, saving model to model_1.1
Epoch 2/30
7352/7352 [==============================] - 238s 32ms/step - loss: 0.6422 - accuracy: 0.7232 - val_loss: 0.7311 - val_accuracy: 0.7150

Epoch 00002: val_accuracy improved from 0.66135 to 0.71496, saving model to model_1.1
Epoch 3/30
7352/7352 [==============================] - 261s 35ms/step - loss: 0.4935 - accuracy: 0.7767 - val_loss: 0.5442 - val_accuracy: 0.7655

Epoch 00003: val_accuracy improved from 0.71496 to 0.76552, saving model to model_1.1
Epoch 4/30
7352/7352 [==============================] - 302s 41ms/step - loss: 0.3849 - accuracy: 0.8541 - val_loss: 0.4581 - val_accuracy: 0.8833

Epoch 00004: val_accuracy improved from 0.76552 to 0.88327, saving model to model_1.1
Epoch 5/30
7352/735

In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 524        0         0        0                   0   
SITTING                  2      462        23        2                   0   
STANDING                 0      220       309        3                   0   
WALKING                  0        0         0      441                  54   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        8                  15   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            13  
SITTING                            2  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 448  


In [25]:
score = model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 12s 4ms/step


[0.41557583960709327, 0.8836104273796082]

In [16]:
# Initiliazing the sequential model
from keras.layers.normalization import BatchNormalization
model = Sequential()
# Configuring the parameters
model.add(LSTM(128, input_shape=(timesteps, input_dim)))
model.add(BatchNormalization())
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               70656     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 71,942
Trainable params: 71,686
Non-trainable params: 256
_________________________________________________________________


In [17]:
from keras.callbacks.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint_1 = ModelCheckpoint("best_model_1.1", monitor="val_accuracy",mode="max",save_best_only = True, verbose=1) 


early_stop=EarlyStopping(monitor='val_accuracy',patience=3)
callbacks = [checkpoint_1,early_stop]

In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [20]:
#Training the model
model.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),
          epochs=20,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 74s 10ms/step - loss: 0.8231 - accuracy: 0.6401 - val_loss: 0.7539 - val_accuracy: 0.6705

Epoch 00001: val_accuracy improved from -inf to 0.67051, saving model to best_model_1.1
Epoch 2/20
7352/7352 [==============================] - 72s 10ms/step - loss: 0.6254 - accuracy: 0.7359 - val_loss: 0.5794 - val_accuracy: 0.7896

Epoch 00002: val_accuracy improved from 0.67051 to 0.78962, saving model to best_model_1.1
Epoch 3/20
7352/7352 [==============================] - 73s 10ms/step - loss: 0.3875 - accuracy: 0.8818 - val_loss: 0.5783 - val_accuracy: 0.8229

Epoch 00003: val_accuracy improved from 0.78962 to 0.82287, saving model to best_model_1.1
Epoch 4/20
7352/7352 [==============================] - 72s 10ms/step - loss: 0.2199 - accuracy: 0.9257 - val_loss: 0.3194 - val_accuracy: 0.9026

Epoch 00004: val_accuracy improved from 0.82287 to 0.90261, saving model to best_model_1.1
Epo

In [21]:
from keras.models import load_model
saved_model = load_model('best_model_1.1')

In [22]:
# Confusion Matrix
print(confusion_matrix(Y_test, saved_model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      405        82        0                   0   
STANDING                 0       69       463        0                   0   
WALKING                  0        1         2      464                  24   
WALKING_DOWNSTAIRS       0        0         0        0                 417   
WALKING_UPSTAIRS         0        1         1        0                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            5  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 468  


In [23]:
score = saved_model.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 13s 4ms/step


[0.24674624278151186, 0.9345096945762634]

In [24]:
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(128, return_sequences=True, input_shape=(128,9)))
# Adding a dropout layer
model1.add(Dropout(0.2))

model1.add(LSTM(64))
# Adding a dropout layer
model1.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [25]:
from keras.callbacks.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint_1 = ModelCheckpoint("best_model_2.1", monitor="val_accuracy",mode="max",save_best_only = True, verbose=1) 


early_stop=EarlyStopping(monitor='val_accuracy',patience=3)
callbacks = [checkpoint_1,early_stop]

In [27]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [28]:
#Training the model
model1.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 106s 14ms/step - loss: 1.0653 - accuracy: 0.5418 - val_loss: 0.8596 - val_accuracy: 0.6308

Epoch 00001: val_accuracy improved from -inf to 0.63081, saving model to best_model_2.1
Epoch 2/30
7352/7352 [==============================] - 72s 10ms/step - loss: 0.7825 - accuracy: 0.6428 - val_loss: 0.9171 - val_accuracy: 0.5925

Epoch 00002: val_accuracy did not improve from 0.63081
Epoch 3/30
7352/7352 [==============================] - 72s 10ms/step - loss: 0.6863 - accuracy: 0.7084 - val_loss: 0.6890 - val_accuracy: 0.7014

Epoch 00003: val_accuracy improved from 0.63081 to 0.70139, saving model to best_model_2.1
Epoch 4/30
7352/7352 [==============================] - 72s 10ms/step - loss: 0.5301 - accuracy: 0.8040 - val_loss: 0.4430 - val_accuracy: 0.8680

Epoch 00004: val_accuracy improved from 0.70139 to 0.86800, saving model to best_model_2.1
Epoch 5/30
7352/7352 [================

In [29]:
from keras.models import load_model
saved_model_1 = load_model('best_model_2.1')

In [30]:
# Confusion Matrix
print(confusion_matrix(Y_test, saved_model_1.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  5      378        99        0                   0   
STANDING                 0       67       464        0                   0   
WALKING                  0        0         0      467                  15   
WALKING_DOWNSTAIRS       0        0         0        1                 412   
WALKING_UPSTAIRS         0        0         0       13                   2   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            9  
STANDING                           1  
WALKING                           14  
WALKING_DOWNSTAIRS                 7  
WALKING_UPSTAIRS                 456  


In [31]:
score = saved_model_1.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 14s 5ms/step


[0.27421482619152027, 0.9209365248680115]

In [36]:
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(128, return_sequences=True, input_shape=(128,9)))
# Adding a dropout layer
model1.add(Dropout(0.2))

model1.add(LSTM(64))
# Adding a dropout layer
model1.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
Total params: 120,454
Trainable params: 120,454
Non-trainable params: 0
_________________________________________________________________


In [37]:
from keras.callbacks.callbacks import EarlyStopping
from keras.callbacks.callbacks import ModelCheckpoint
checkpoint_1 = ModelCheckpoint("best_model_3.1", monitor="val_accuracy",mode="max",save_best_only = True, verbose=1) 


early_stop=EarlyStopping(monitor='val_accuracy',patience=5)
callbacks = [checkpoint_1,early_stop]

In [38]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [39]:
#Training the model
model1.fit(X_train,
          Y_train,
          batch_size=32,
          validation_data=(X_test, Y_test),
          epochs=epochs,callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 71s 10ms/step - loss: 1.0782 - accuracy: 0.5335 - val_loss: 0.8807 - val_accuracy: 0.6040

Epoch 00001: val_accuracy improved from -inf to 0.60400, saving model to best_model_3.1
Epoch 2/30
7352/7352 [==============================] - 71s 10ms/step - loss: 0.7853 - accuracy: 0.6530 - val_loss: 0.7115 - val_accuracy: 0.7095

Epoch 00002: val_accuracy improved from 0.60400 to 0.70954, saving model to best_model_3.1
Epoch 3/30
7352/7352 [==============================] - 71s 10ms/step - loss: 0.6406 - accuracy: 0.7206 - val_loss: 0.5805 - val_accuracy: 0.7523

Epoch 00003: val_accuracy improved from 0.70954 to 0.75229, saving model to best_model_3.1
Epoch 4/30
7352/7352 [==============================] - 71s 10ms/step - loss: 0.4090 - accuracy: 0.8376 - val_loss: 0.4350 - val_accuracy: 0.8809

Epoch 00004: val_accuracy improved from 0.75229 to 0.88090, saving model to best_model_3.1
Epo

In [40]:
from keras.models import load_model
saved_model_2 = load_model('best_model_3.1')

In [41]:
# Confusion Matrix
print(confusion_matrix(Y_test, saved_model_2.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                 24      387        77        0                   0   
STANDING                 0       61       470        0                   0   
WALKING                  0        0         0      454                  40   
WALKING_DOWNSTAIRS       0        0         0        0                 419   
WALKING_UPSTAIRS         0        1         0        1                   6   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           1  
WALKING                            2  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 463  


In [42]:
score = saved_model_2.evaluate(X_test, Y_test)
score

2947/2947 [==============================] - 14s 5ms/step


[0.3025784575979611, 0.9263657927513123]

# RESULTS

In [117]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ['S.NO',"No. of Hidden Layers", "No.of lstm units", "Dropout", "TEST ACCURACY",'TEST LOSS']
x.add_row(["1","1",'32','0.25','0.8948','0.3549'])
x.add_row(["2","2",'32','0.75','0.6244','0.7772'])
x.add_row(["3","2",'20','0.5','0.9040','0.4683'])
x.add_row(["4","2",'24','0.5','0.9016','0.3812'])
x.add_row(["5","2",'28','0.5','0.7234','0.7294'])
x.add_row(["6","2",'28','0.75','0.5205','0.8599'])
x.add_row(["7","2",'28','0.25','0.9074','0.4082'])

print(x)

+------+----------------------+------------------+---------+---------------+-----------+
| S.NO | No. of Hidden Layers | No.of lstm units | Dropout | TEST ACCURACY | TEST LOSS |
+------+----------------------+------------------+---------+---------------+-----------+
|  1   |          1           |        32        |   0.25  |     0.8948    |   0.3549  |
|  2   |          2           |        32        |   0.75  |     0.6244    |   0.7772  |
|  3   |          2           |        20        |   0.5   |     0.9040    |   0.4683  |
|  4   |          2           |        24        |   0.5   |     0.9016    |   0.3812  |
|  5   |          2           |        28        |   0.5   |     0.7234    |   0.7294  |
|  6   |          2           |        28        |   0.75  |     0.5205    |   0.8599  |
|  7   |          2           |        28        |   0.25  |     0.9074    |   0.4082  |
+------+----------------------+------------------+---------+---------------+-----------+


# CONCLUSION

With 1 hidden layer with 128 lstm units  and dropout rate of 0.25 we got test accuracy of 93.45% and test loss of 0.2467.
With 2 hidden layers with 128 and 64 lstm units and dropout rates of 0.2 and 0.5 respectively we got test accuracy of 92.63% and
test loss of 0.3025. 